### Aim of the notebook is to visualize the receptive fields of convolution filters 

This notebook is simplified version of github repo [https://github.com/metalbubble/cnnvisualizer](https://github.com/metalbubble/cnnvisualizer)

Parts of codes have been used for the ICLR'15 paper Object Detectors Emerge in Deep Scene CNN

In [12]:
import torch
from torch.autograd import Variable as V
from torchvision import transforms as trn
import os
import numpy as np
import cv2
from dataset import Dataset
import torchvision.models as models
import torch.utils.data as data
import torchvision.models as models

In [13]:
# visualization setup
img_size = (227, 227)       # input image size
segment_size = (120,120)    # the unit segmentaiton size
num_top = 12                # how many top activated images to extract
margin = 3                  # pixels between two segments
threshold_scale = 0.2       # the scale used to segment the feature map. Smaller the segmentation will be tighter.

In [14]:
# dataset setup
batch_size = 64
num_workers = 6

In [22]:
# Load required model
model_name = 'alexnetimagenet'
model = models.alexnet(pretrained=True)
#model = models.resnet18(pretrained=True)
features_names = ['layer4']

In [23]:
model.eval()
model.cuda()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [17]:
### Download the image dataset
!sh download_images.sh

--2018-12-09 02:23:27--  http://places.csail.mit.edu/unit_annotation/data/images.tar
Resolving places.csail.mit.edu (places.csail.mit.edu)... 128.30.100.255
Connecting to places.csail.mit.edu (places.csail.mit.edu)|128.30.100.255|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 755445760 (720M) [application/x-tar]
Saving to: ‘images.tar’

images.tar          100%[===================>] 720.45M  19.4MB/s    in 93s     

2018-12-09 02:25:01 (7.71 MB/s) - ‘images.tar’ saved [755445760/755445760]

images/
images/readme.txt
images/imagenet/
images/imagenet/012988.jpg
images/imagenet/009763.jpg
images/imagenet/005704.jpg
images/imagenet/017503.jpg
images/imagenet/017971.jpg
images/imagenet/020938.jpg
images/imagenet/004144.jpg
images/imagenet/016896.jpg
images/imagenet/008123.jpg
images/imagenet/008684.jpg
images/imagenet/016343.jpg
images/imagenet/024454.jpg
images/imagenet/024826.jpg
images/imagenet/007051.jpg
images/imagenet/019596.jpg
images/imagenet/015983.jpg
ima

images/imagenet/015642.jpg
images/imagenet/019625.jpg
images/imagenet/007494.jpg
images/imagenet/019153.jpg
images/imagenet/015134.jpg
images/imagenet/007333.jpg
images/imagenet/015693.jpg
images/imagenet/014774.jpg
images/imagenet/018713.jpg
images/imagenet/006901.jpg
images/imagenet/006573.jpg
images/imagenet/020628.jpg
images/imagenet/017661.jpg
images/imagenet/012698.jpg
images/imagenet/009873.jpg
images/imagenet/009401.jpg
images/imagenet/000338.jpg
images/imagenet/005466.jpg
images/imagenet/005814.jpg
images/imagenet/004581.jpg
images/imagenet/016021.jpg
images/imagenet/021068.jpg
images/imagenet/008994.jpg
images/imagenet/024736.jpg
images/imagenet/004226.jpg
images/imagenet/001578.jpg
images/imagenet/008241.jpg
images/imagenet/016786.jpg
images/imagenet/024091.jpg
images/imagenet/014315.jpg
images/imagenet/011839.jpg
images/imagenet/018372.jpg
images/imagenet/023889.jpg
images/imagenet/006112.jpg
images/imagenet/019940.jpg
images/imagenet/019532.jpg
images/imagenet/015555.jpg
i

images/imagenet/007805.jpg
images/imagenet/010689.jpg
images/imagenet/019617.jpg
images/imagenet/022639.jpg
images/imagenet/015670.jpg
images/imagenet/014797.jpg
images/imagenet/006237.jpg
images/imagenet/003569.jpg
images/imagenet/014030.jpg
images/imagenet/023079.jpg
images/imagenet/006590.jpg
images/imagenet/018057.jpg
images/imagenet/004103.jpg
images/imagenet/008164.jpg
images/imagenet/021898.jpg
images/imagenet/016304.jpg
images/imagenet/013828.jpg
images/imagenet/024413.jpg
images/imagenet/024861.jpg
images/imagenet/009724.jpg
images/imagenet/005743.jpg
images/imagenet/009083.jpg
images/imagenet/017544.jpg
images/imagenet/017936.jpg
images/imagenet/003108.jpg
images/imagenet/006656.jpg
images/imagenet/018391.jpg
images/imagenet/023418.jpg
images/imagenet/014823.jpg
images/imagenet/014451.jpg
images/imagenet/018436.jpg
images/imagenet/018844.jpg
images/imagenet/007016.jpg
images/imagenet/002748.jpg
images/imagenet/019276.jpg
images/imagenet/015211.jpg
images/imagenet/022258.jpg
i

images/imagenet/005350.jpg
images/imagenet/009337.jpg
images/imagenet/006990.jpg
images/imagenet/018025.jpg
images/imagenet/014042.jpg
images/imagenet/018782.jpg
images/imagenet/006245.jpg
images/imagenet/003969.jpg
images/imagenet/015602.jpg
images/imagenet/019665.jpg
images/imagenet/007405.jpg
images/imagenet/007877.jpg
images/imagenet/005096.jpg
images/imagenet/012268.jpg
images/imagenet/017536.jpg
images/imagenet/017944.jpg
images/imagenet/005731.jpg
images/imagenet/017291.jpg
images/imagenet/009756.jpg
images/imagenet/016376.jpg
images/imagenet/013428.jpg
images/imagenet/024461.jpg
images/imagenet/024813.jpg
images/imagenet/001188.jpg
images/imagenet/008116.jpg
images/imagenet/021498.jpg
images/imagenet/004171.jpg
images/imagenet/015263.jpg
images/imagenet/019204.jpg
images/imagenet/007064.jpg
images/imagenet/018444.jpg
images/imagenet/018836.jpg
images/imagenet/006183.jpg
images/imagenet/023818.jpg
images/imagenet/014851.jpg
images/imagenet/014423.jpg
images/imagenet/006624.jpg
i

images/imagenet/016659.jpg
images/imagenet/021610.jpg
images/imagenet/008439.jpg
images/imagenet/001300.jpg
images/imagenet/010012.jpg
images/imagenet/022705.jpg
images/imagenet/002215.jpg
images/imagenet/007939.jpg
images/imagenet/023145.jpg
images/imagenet/011652.jpg
images/imagenet/003455.jpg
images/imagenet/003827.jpg
images/imagenet/021271.jpg
images/imagenet/016238.jpg
images/imagenet/013566.jpg
images/imagenet/013914.jpg
images/imagenet/004798.jpg
images/imagenet/008058.jpg
images/imagenet/024288.jpg
images/imagenet/001761.jpg
images/imagenet/000686.jpg
images/imagenet/012326.jpg
images/imagenet/017478.jpg
images/imagenet/020431.jpg
images/imagenet/020843.jpg
images/imagenet/000121.jpg
images/imagenet/012481.jpg
images/imagenet/009618.jpg
images/imagenet/020396.jpg
images/imagenet/003793.jpg
images/imagenet/018978.jpg
images/imagenet/011233.jpg
images/imagenet/023956.jpg
images/imagenet/023524.jpg
images/imagenet/003034.jpg
images/imagenet/011594.jpg
images/imagenet/023283.jpg
i

...

images/sun/026677.jpg
images/sun/023129.jpg
images/sun/014160.jpg
images/sun/018107.jpg
images/sun/011199.jpg
images/sun/003439.jpg
images/sun/006367.jpg
images/sun/025762.jpg
images/sun/017075.jpg
images/sun/009215.jpg
images/sun/005272.jpg
images/sun/004395.jpg
images/sun/016635.jpg
images/sun/024122.jpg
images/sun/004432.jpg
images/sun/004840.jpg
images/sun/008827.jpg
images/sun/008455.jpg
images/sun/016192.jpg
images/sun/024685.jpg
images/sun/008323.jpg
images/sun/004344.jpg
images/sun/001868.jpg
images/sun/016143.jpg
images/sun/008484.jpg
images/sun/024654.jpg
images/sun/004891.jpg
images/sun/005504.jpg
images/sun/005976.jpg
images/sun/009911.jpg
images/sun/009563.jpg
images/sun/025014.jpg
images/sun/017703.jpg
images/sun/006863.jpg
images/sun/006411.jpg
images/sun/018671.jpg
images/sun/026101.jpg
images/sun/011148.jpg
images/sun/014616.jpg
images/sun/019796.jpg
images/sun/007251.jpg
images/sun/015056.jpg
images/sun/010708.jpg
images/sun/007984.jpg
images/sun/019031.jpg
images/sun

images/sun/015970.jpg
images/sun/019917.jpg
images/sun/019565.jpg
images/sun/018482.jpg
images/sun/006145.jpg
images/sun/014897.jpg
images/sun/018325.jpg
images/sun/026455.jpg
images/sun/026827.jpg
images/sun/014342.jpg
images/sun/005050.jpg
images/sun/009037.jpg
images/sun/017982.jpg
images/sun/012509.jpg
images/sun/025932.jpg
images/sun/025540.jpg
images/sun/017257.jpg
images/sun/009790.jpg
images/sun/008677.jpg
images/sun/004610.jpg
images/sun/016865.jpg
images/sun/016417.jpg
images/sun/024300.jpg
images/sun/013349.jpg
images/sun/016689.jpg
images/sun/004329.jpg
images/sun/001477.jpg
images/sun/001805.jpg
images/sun/021167.jpg
images/sun/013670.jpg
images/sun/024639.jpg
images/sun/000237.jpg
images/sun/005569.jpg
images/sun/012797.jpg
images/sun/020080.jpg
images/sun/000590.jpg
images/sun/025079.jpg
images/sun/012030.jpg
images/sun/020727.jpg
images/sun/003322.jpg
images/sun/011682.jpg
images/sun/023195.jpg
images/sun/003485.jpg
images/sun/011125.jpg
images/sun/023632.jpg
images/sun

images/sun/009042.jpg
images/sun/017585.jpg
images/sun/025292.jpg
images/sun/005025.jpg
images/sun/017222.jpg
images/sun/025535.jpg
images/sun/025947.jpg
images/sun/005782.jpg
images/sun/004665.jpg
images/sun/008602.jpg
images/sun/024375.jpg
images/sun/021859.jpg
images/sun/016462.jpg
images/sun/016810.jpg
images/sun/001870.jpg
images/sun/001402.jpg
images/sun/004889.jpg
images/sun/013605.jpg
images/sun/021112.jpg
images/sun/009909.jpg
images/sun/000242.jpg
images/sun/020752.jpg
images/sun/012045.jpg
images/sun/000997.jpg
images/sun/006409.jpg
images/sun/003357.jpg
images/sun/023647.jpg
images/sun/026119.jpg
images/sun/011150.jpg
images/sun/018669.jpg
images/sun/003882.jpg
images/sun/002517.jpg
images/sun/002965.jpg
images/sun/007249.jpg
images/sun/019029.jpg
images/sun/010710.jpg
images/sun/022007.jpg
images/sun/012383.jpg
images/sun/020494.jpg
images/sun/000623.jpg
images/sun/012424.jpg
images/sun/012856.jpg
images/sun/020333.jpg
images/sun/000184.jpg
images/sun/001063.jpg
images/sun

images/sun/023027.jpg
images/sun/026779.jpg
images/sun/011730.jpg
images/sun/018009.jpg
images/sun/003290.jpg
images/sun/002377.jpg
images/sun/007429.jpg
images/sun/015189.jpg
images/sun/019649.jpg
images/sun/010170.jpg
images/sun/027139.jpg
images/sun/022667.jpg
images/sun/001262.jpg
images/sun/008929.jpg
images/sun/013065.jpg
images/sun/021772.jpg
images/sun/020695.jpg
images/sun/012182.jpg
images/sun/000850.jpg
images/sun/000422.jpg
images/sun/020132.jpg
images/sun/012625.jpg
images/sun/000385.jpg
images/sun/007048.jpg
images/sun/002716.jpg
images/sun/019228.jpg
images/sun/022206.jpg
images/sun/010963.jpg
images/sun/010511.jpg
images/sun/011884.jpg
images/sun/003156.jpg
images/sun/006608.jpg
images/sun/011351.jpg
images/sun/026318.jpg
images/sun/023834.jpg
images/sun/023446.jpg
images/sun/018468.jpg
images/sun/012991.jpg
images/sun/000043.jpg
images/sun/012244.jpg
images/sun/017968.jpg
images/sun/020553.jpg
images/sun/020921.jpg
images/sun/001603.jpg
images/sun/021313.jpg
images/sun

images/sun/005251.jpg
images/sun/009236.jpg
images/sun/005984.jpg
images/sun/009591.jpg
images/sun/017056.jpg
images/sun/025741.jpg
images/sun/012708.jpg
images/sun/006344.jpg
images/sun/018683.jpg
images/sun/003868.jpg
images/sun/018124.jpg
images/sun/006891.jpg
images/sun/014143.jpg
images/sun/026654.jpg
images/sun/007976.jpg
images/sun/007504.jpg
images/sun/027014.jpg
images/sun/015703.jpg
images/sun/019764.jpg
images/sun/005630.jpg
images/sun/025487.jpg
images/sun/009657.jpg
images/sun/017390.jpg
images/sun/005197.jpg
images/sun/012369.jpg
images/sun/025320.jpg
images/sun/017845.jpg
images/sun/017437.jpg
images/sun/021599.jpg
images/sun/008017.jpg
images/sun/004070.jpg
images/sun/016277.jpg
images/sun/024912.jpg
images/sun/024560.jpg
images/sun/013529.jpg
images/sun/001089.jpg
images/sun/007165.jpg
images/sun/015362.jpg
images/sun/019305.jpg
images/sun/006725.jpg
images/sun/026592.jpg
images/sun/014285.jpg
images/sun/006082.jpg
images/sun/018937.jpg
images/sun/018545.jpg
images/sun

In [24]:
# image datasest to be processed
name_dataset = 'sun+imagenetaval'
root_image = 'images'
with open('images/imagelist.txt') as f:
    lines = f.readlines()
imglist = []
for line in lines:
    line = line.rstrip()
    imglist.append(root_image + '/' + line)

In [25]:
features_blobs = []
def hook_feature(module, input, output):
    # hook the feature extractor
    features_blobs.append(np.squeeze(output.data.cpu().numpy()))

#for name in features_names:
#model._modules.get(name).register_forward_hook(hook_feature)
layer = list(model.features.children())[8]
layer.register_forward_hook(hook_feature)

In [26]:
# image transformer
tf = trn.Compose([
        trn.Resize(img_size),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [27]:
dataset = Dataset(imglist, tf)
loader = data.DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False)

In [28]:
# extract the max value activaiton for each image
imglist_results = []
maxfeatures = [None] * len(features_names)
num_batches = len(dataset) / batch_size
for batch_idx, (input, paths) in enumerate(loader):
    del features_blobs[:]
    print('%d / %d' % (batch_idx+1, num_batches))
    input = input.cuda()
    input_var = V(input, volatile=True)
    logit = model.forward(input_var)
    imglist_results = imglist_results + list(paths)
    if maxfeatures[0] is None:
        # initialize the feature variable
        for i, feat_batch in enumerate(features_blobs):
            size_features = (len(dataset), feat_batch.shape[1])
            maxfeatures[i] = np.zeros(size_features)
    start_idx = batch_idx*batch_size
    end_idx = min((batch_idx+1)*batch_size, len(dataset))
    for i, feat_batch in enumerate(features_blobs):
        maxfeatures[i][start_idx:end_idx] = np.max(np.max(feat_batch,3),2)

1 / 815
2 / 815
3 / 815


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if __name__ == '__main__':


4 / 815
5 / 815
6 / 815
7 / 815
8 / 815
9 / 815
10 / 815
11 / 815
12 / 815
13 / 815
14 / 815
15 / 815
16 / 815
17 / 815
18 / 815
19 / 815
20 / 815
21 / 815
22 / 815
23 / 815
24 / 815
25 / 815
26 / 815
27 / 815
28 / 815
29 / 815
30 / 815
31 / 815
32 / 815
33 / 815
34 / 815
35 / 815
36 / 815
37 / 815
38 / 815
39 / 815
40 / 815
41 / 815
42 / 815
43 / 815
44 / 815
45 / 815
46 / 815
47 / 815
48 / 815
49 / 815
50 / 815
51 / 815
52 / 815
53 / 815
54 / 815
55 / 815
56 / 815
57 / 815
58 / 815
59 / 815
60 / 815
61 / 815
62 / 815
63 / 815
64 / 815
65 / 815
66 / 815
67 / 815
68 / 815
69 / 815
70 / 815
71 / 815
72 / 815
73 / 815
74 / 815
75 / 815
76 / 815
77 / 815
78 / 815
79 / 815
80 / 815
81 / 815
82 / 815
83 / 815
84 / 815
85 / 815
86 / 815
87 / 815
88 / 815
89 / 815
90 / 815
91 / 815
92 / 815
93 / 815
94 / 815
95 / 815
96 / 815
97 / 815
98 / 815
99 / 815
100 / 815
101 / 815
102 / 815
103 / 815
104 / 815
105 / 815
106 / 815
107 / 815
108 / 815
109 / 815
110 / 815
111 / 815
112 / 815
113 / 815
11

In [29]:
# generate the top activated images
output_folder = 'result_segments/%s' % model_name
if not os.path.exists(output_folder):
    os.makedirs(output_folder + '/image')

In [30]:
# output the html first
for layerID, layer in enumerate(features_names):
    file_html = os.path.join(output_folder, layer + '.html')
    with open(file_html, 'w') as f:
        num_units = maxfeatures[layerID].shape[1]
        lines_units = ['%s-unit%03d.jpg' % (layer, unitID) for unitID in range(num_units)]
        lines_units = ['unit%03d<br><img src="image/%s">'%(unitID, lines_units[unitID]) for unitID in range(num_units)]
        f.write('\n<br>'.join(lines_units))

In [31]:
# generate the unit visualization
for layerID, layer in enumerate(features_names):
    num_units = maxfeatures[layerID].shape[1]
    imglist_sorted = []
    # load the top actiatied image list into one list
    for unitID in range(num_units):
        activations_unit = np.squeeze(maxfeatures[layerID][:, unitID])
        idx_sorted = np.argsort(activations_unit)[::-1]
        imglist_sorted += [imglist[item] for item in idx_sorted[:num_top]]

    # data loader for the top activated images
    loader_top = data.DataLoader(
        Dataset(imglist_sorted, tf),
        batch_size=num_top,
        num_workers=num_workers,
        shuffle=False)
    for unitID, (input, paths) in enumerate(loader_top):
        del features_blobs[:]
        print('%d / %d' % (unitID+1, num_units))
        input = input.cuda()
        input_var = V(input, volatile=True)
        logit = model.forward(input_var)
        feature_maps = features_blobs[layerID]

        images_input = input.cpu().numpy()
        max_value = 0
        output_unit = []
        for i in range(num_top):
            feature_map = feature_maps[i][unitID]
            if max_value == 0:
                max_value = np.max(feature_map)
            feature_map = feature_map / max_value
            mask = cv2.resize(feature_map, segment_size)
            mask[mask < threshold_scale] = 0.0 # binarize the mask
            mask[mask > threshold_scale] = 1.0

            img = cv2.imread(paths[i])
            img = cv2.resize(img, segment_size)
            img = cv2.normalize(img.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
            img_mask = np.multiply(img, mask[:,:, np.newaxis])
            img_mask = np.uint8(img_mask * 255)
            output_unit.append(img_mask)
            output_unit.append(np.uint8(np.ones((segment_size[0],margin,3))*255))
        montage_unit = np.concatenate(output_unit, axis=1)
        cv2.imwrite(os.path.join(output_folder, 'image', '%s-unit%03d.jpg'%(layer, unitID)), montage_unit)
print('done check results in ' + output_folder)

1 / 256


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


2 / 256
3 / 256
4 / 256
5 / 256
6 / 256
7 / 256
8 / 256
9 / 256
10 / 256
11 / 256
12 / 256
13 / 256
14 / 256
15 / 256
16 / 256
17 / 256
18 / 256
19 / 256
20 / 256
21 / 256
22 / 256
23 / 256
24 / 256
25 / 256
26 / 256
27 / 256
28 / 256
29 / 256
30 / 256
31 / 256
32 / 256
33 / 256
34 / 256
35 / 256
36 / 256
37 / 256
38 / 256
39 / 256
40 / 256
41 / 256
42 / 256
43 / 256
44 / 256
45 / 256
46 / 256
47 / 256
48 / 256
49 / 256
50 / 256
51 / 256
52 / 256
53 / 256
54 / 256
55 / 256
56 / 256
57 / 256
58 / 256
59 / 256
60 / 256
61 / 256
62 / 256
63 / 256
64 / 256
65 / 256
66 / 256
67 / 256
68 / 256
69 / 256
70 / 256
71 / 256
72 / 256
73 / 256
74 / 256
75 / 256
76 / 256
77 / 256
78 / 256
79 / 256
80 / 256
81 / 256
82 / 256
83 / 256
84 / 256
85 / 256
86 / 256
87 / 256
88 / 256
89 / 256
90 / 256
91 / 256
92 / 256
93 / 256
94 / 256
95 / 256
96 / 256
97 / 256
98 / 256
99 / 256
100 / 256
101 / 256
102 / 256
103 / 256
104 / 256
105 / 256
106 / 256
107 / 256
108 / 256
109 / 256
110 / 256
111 / 256
112 / 